<a href="https://colab.research.google.com/github/nbhami2/GNN_Model/blob/main/GNN_Model_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch_geometric
!pip install mne
!pip install mne-connectivity
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader
import numpy as np
import networkx as nx
from sklearn.model_selection import train_test_split

import os
import mne
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset, WeightedRandomSampler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,ConfusionMatrixDisplay,roc_curve, auc, mutual_info_score
import matplotlib.pyplot as plt
from graphviz import Digraph
import mne_connectivity
import seaborn as sns
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 66.5 MB/s eta 0:00:00


In [ ]:
data_dir = '/content/drive/MyDrive/gnn_data'

In [ ]:
raw_data_path = os.path.join(data_dir, 'correlation_data.npz')
data_fd = np.load(raw_data_path)  # Ensure this is defined
X = data_fd['X']
y = data_fd['y']

chs = [
     'C1','C3', 'C5', 'CP5', 'CP1', 'CP6', 'CP2',  'CP3', 'CP4', 'C6', 'C2', 'C4'
      ]

# Define FM scale classification cls
cls = {
    '004C': 0, '005C': 0, '006C': 0, '007C': 0, '008C': 0, '009C': 0, '023C': 0, '024C': 0, '025C': 0,
    '027C': 0, '029C': 0, '030C': 0, '031C': 0, '032C': 0, # Class 1 Normal
    '002S': 1, '011S': 1, '012S': 1, '013S': 1, '015S': 1, '021S': 1, '034S': 1, # Class 2: FM 30-45
    '001S': 2, '016S': 2, '018S': 2, '019S': 2, '033S': 2  # Class 3: FM < 30 (Severe impairment)
}

In [ ]:
# EEG channel names (should match the number of channels in your data)
chs = [
     'C1','C3', 'C5', 'CP5', 'CP1', 'CP6', 'CP2',  'CP3', 'CP4', 'C6', 'C2', 'C4'
      ]

In [ ]:
class GNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim, normalize=True)
        self.conv2 = GCNConv(hidden_dim, output_dim, normalize=True)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = F.relu(self.conv1(x, edge_index, edge_weight=edge_attr))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index, edge_weight=edge_attr)
        return F.log_softmax(x, dim=-1)

In [ ]:
# def create_eeg_graph(num_channels=12, features_dim=5, label=None):
#     node_features = torch.rand((num_channels, features_dim), dtype=torch.float)

#     connectivity_matrix = np.random.rand(num_channels, num_channels)
#     connectivity_matrix = (connectivity_matrix + connectivity_matrix.T) / 2
#     np.fill_diagonal(connectivity_matrix, 0)

#     edge_indices = np.nonzero(connectivity_matrix)
#     edge_index = torch.tensor(np.array(edge_indices), dtype=torch.long)

#     edge_weights = torch.tensor(connectivity_matrix[edge_indices], dtype=torch.float)
#     node_labels = torch.tensor([label] * num_channels, dtype=torch.long)
#     return Data(x=node_features, edge_index=edge_index, edge_attr=edge_weights, y=node_labels)


# # Create multiple graphs for dataset
# num_samples = 20
# labels = torch.tensor(np.random.randint(0, 2, num_samples), dtype=torch.long)  # Binary labels
# graphs = [create_eeg_graph(label=labels[i]) for i in range(num_samples)]  # Pass the label to create_eeg_graph


# train_graphs, val_graphs, train_labels, val_labels = train_test_split(graphs, labels, test_size=0.2, random_state=42)

labels = torch.tensor(y, dtype=torch.long)
graphs = torch.tensor(X, dtype=torch.float)
train_graphs, val_graphs, train_labels, val_labels = train_test_split(graphs, labels, test_size=0.2, random_state=42)

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader
import numpy as np
import networkx as nx
from sklearn.model_selection import train_test_split

graphs = []
for i in range(X.shape[0]):
    num_nodes = X.shape[1]
    edge_index = torch.tensor(
        [[i, j] for i in range(num_nodes) for j in range(num_nodes) if i != j],
        dtype=torch.long
    ).t().contiguous()

    edge_attr = torch.ones(edge_index.shape[1], dtype=torch.float)

    data = Data(x=torch.tensor(X[i], dtype=torch.float),
                edge_index=edge_index,
                edge_attr=edge_attr,
                y=torch.tensor(y[i], dtype=torch.long))
    graphs.append(data)

# Split the data
train_graphs, val_graphs, train_labels, val_labels = train_test_split(graphs, labels, test_size=0.2, random_state=42)

# Create DataLoaders
train_loader = DataLoader(train_graphs, batch_size=16, shuffle=True)
val_loader = DataLoader(val_graphs, batch_size=16, shuffle=False)


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
train_loader = DataLoader(train_graphs, batch_size=16, shuffle=True)
val_loader = DataLoader(val_graphs, batch_size=16, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GNN(input_dim=X.shape[2], hidden_dim=16, output_dim=5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.NLLLoss()

def train(model, loader, optimizer, criterion, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0
        for data in loader:
            data = data.to(device)
            optimizer.zero_grad()
            output = model(data)
            target = data.y[0].repeat(output.shape[0])
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            preds = output.argmax(dim=1)
            correct += (preds == target).sum().item()
            total += target.size(0)
        accuracy = correct / total
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(loader):.4f}, Accuracy: {accuracy:.4f}")

train(model, train_loader, optimizer, criterion)

/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1, Loss: 1.2953, Accuracy: 0.5692
Epoch 2, Loss: 1.2495, Accuracy: 0.5716
Epoch 3, Loss: 1.2274, Accuracy: 0.5653
Epoch 4, Loss: 1.3012, Accuracy: 0.5327
Epoch 5, Loss: 1.2116, Accuracy: 0.5741
Epoch 6, Loss: 1.1908, Accuracy: 0.5942
Epoch 7, Loss: 1.1947, Accuracy: 0.5879
Epoch 8, Loss: 1.2454, Accuracy: 0.5666
Epoch 9, Loss: 1.2252, Accuracy: 0.5729
Epoch 10, Loss: 1.1703, Accuracy: 0.5992


In [ ]:
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            output = model(data)
            preds = output.argmax(dim=1)
            targets = data.y[0].repeat(output.shape[0])
            loss = criterion(output, targets)
            correct += (preds == targets).sum().item()
            total += targets.size(0)
    print(f"Validation Accuracy: {correct / total:.2%}")
    print(f"Validation Loss: {loss.item():.4f}")

evaluate(model, val_loader)